In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Send the data to the sheet

In [12]:
sql = 'select * from crowdsdb.dbo.vw_consolidated_socialdistancing'

In [13]:
consolidated_sql = (pd.read_sql(con = engine, sql = sql)
                    .fillna(value = np.nan, axis = 1))

In [14]:
consolidated_sql.tail()

,source_survey,encounter_datetime,gispropnum,reported_as,site_id,site_desc,park_borough,location_adddesc,city_agency,encounter_type,amenity,patroncount,police_precinct,police_boro_com
27490,DPR Crowds,2020-03-25 10:54:28,R005,R005,R005-ZN01,Clove Lakes Park Zone 1,Staten Island,NaN,DPR,NaN,NaN,10.0,120,Staten Island
27491,DPR Crowds,2020-03-25 10:49:53,R005,R005,R005,Clove Lakes Park,Staten Island,NaN,DPR,NaN,NaN,6.0,120,Staten Island
27492,DPR Crowds,2020-03-25 10:49:03,R045,R045,R045-01,Schmul Park-Schmul Park,Staten Island,NaN,DPR,NaN,NaN,8.0,122,Staten Island
27493,DPR Crowds,2020-03-25 10:44:20,NaN,NaN,R030-99,NaN,NaN,NaN,DPR,NaN,NaN,18.0,NaN,NaN
27494,DPR Crowds,2020-03-25 08:07:40,M108T01,M108T01,M108T01,East River Esplanade,Manhattan,NaN,DPR,NaN,NaN,17.0,19,Manhattan North


In [15]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [16]:
sheet = client.open('consolidated_social_distancing_data')

In [17]:
ws = sheet.worksheet('Sheet1')

In [18]:
set_with_dataframe(ws, consolidated_sql, include_index = False, 
                   include_column_header = True, resize = True, allow_formulas = False)